In [9]:
# Imports as always
import numpy as np
import sympy
import cirq
import qsimcirq
from scipy.optimize import minimize

from sqif_algorithm import SQIF

In [5]:
# Instantiate an algorithm instance.
solver = SQIF(1997)

# Generate the CVP.
B, t = solver.generate_CVP(c=1.5)

# Classical approximation to the CVP.
B, t, D, b_op, residual_vector, step_signs = solver.find_b_op(B, t)

# Define the Hamiltonian according to the unit hypercube search problem around the approximate solution.
H = solver.define_hamiltonian(D, residual_vector, step_signs)

In [12]:
# Building the circuit.
p = 1

# Get references to the qubits used in the Hamiltonian's definition.
qubits = H.qubits

# Sub-circuit implementing the gamma unitary operator for the i-th layer.
def generate_gamma_operator(i):
    # Define the gamma symbol (as a placeholder).
    gamma = sympy.Symbol(f'gamma_{i}')
    
    # Define the Pauli operators as dense Pauli strings (for recognising terms).
    Z = cirq.DensePauliString('Z')
    ZZ = cirq.DensePauliString('ZZ')
    I = cirq.DensePauliString('')
    
    # Consider each term in the Hamiltonian.
    for term in H:
        # Get the term's operator.
        term_operator = term.with_coefficient(1).gate
        
        # Determine which circuit translation to make, based on which operator this term has.
        if term_operator == Z:
            yield cirq.Z(*term.qubits) ** (gamma * term.coefficient)
        elif term_operator == ZZ:
            yield cirq.ZZ(*term.qubits) ** (gamma * term.coefficient)
        elif term_operator == I:
            yield []
            
        # If the term's operator is unrecognised, the Hamiltonian isn't quite right.
        else:
            raise Exception(f'Unrecognised term in H: {term}')
        
# Sub-circuit implementing the beta unitary ("mixing") operator for the i-th layer.
def generate_beta_operator(i):
    # Define the beta symbol (as a placeholder).
    beta = sympy.Symbol(f'beta_{i}')
    
    return [cirq.X(q) ** beta for q in qubits]

# Generate the circuit for QAOA.
qaoa_circuit = cirq.Circuit(
    # Preparation of uniform superposition.
    cirq.H.on_each(*qubits),
    
    # p layers of repeated U(gamma, H) and U(beta, B) operations.
    [(generate_gamma_operator(i), cirq.Moment(generate_beta_operator(i)),) for i in range(p)],
)

display(qaoa_circuit)

┌────────────────────────────────┐
0: ───H───ZZ───────────────────ZZ──────────────────────────────────Z^(-1.5*gamma_0)──────────────────────X^beta_0───
          │                    │
1: ───H───ZZ^(-4.0*gamma_0)────┼───────────────Z^(-3.5*gamma_0)───────────────────────ZZ─────────────────X^beta_0───
                               │                                                      │
2: ───H────────────────────────ZZ^(2.5*gamma_0)────────────────────Z^(-4.0*gamma_0)───ZZ^(3.0*gamma_0)───X^beta_0───
                              └────────────────────────────────┘

In [13]:
# Finding the optimal parameters for the circuit.

# Get the parameters and observables out of the circuit.
parameter_names = sorted(cirq.parameter_names(qaoa_circuit))
observables = [term.with_coefficient(1) for term in H]

# Define the function to minimise.
def func_to_minimise(x):
    # Define a "parameter resolver" object, which we can use to assign values to parameters.
    parameter_resolver = cirq.ParamResolver(
        # Map each parameter to a value (i.e. assign the circuit's parameters).
        {parameter: value for parameter, value in zip(parameter_names, x)}
    )
    
    # Define a circuit simulator.
    simulator = qsimcirq.QSimSimulator(
        # Pass in some options for the simulator object -- this needs another object, it seems.
        qsimcirq.QSimOptions(
             # Cannot use GPU without compiling qsim locally -- maybe another time.
             use_gpu=False,
             # My PC has 8 cores, so set 8 threads.
             cpu_threads=8,
             verbosity=0
        )
    )
    
    # Simulate the expectation value.
    expectation = simulator.simulate_expectation_values(
        program=qaoa_circuit, observables=observables, param_resolver=parameter_resolver
    )
    
    # Compute the return.
    return sum(term.coefficient * value for term, value in zip(H, expectation)).real

# Let our initial guess be all zeros -- we don't know any better.
x_initial = np.asarray([.0] * len(parameter_names))

# Find the parameters values that minimise the expectation value.
optimal_parameter_values = minimize(func_to_minimise, x_initial, method='BFGS')
display(optimal_parameter_values)

# Stick the parameter values in a parameter resolver object.
optimal_parameters = cirq.ParamResolver(
    {parameter: value for parameter, value in zip(parameter_names, optimal_parameter_values.x)}
)
print(f'\nOptimal parameters: {optimal_parameters}')

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 43.5
        x: [ 0.000e+00  0.000e+00]
      nit: 0
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[1 0]
            [0 1]]
     nfev: 3
     njev: 1


Optimal parameters: cirq.ParamResolver({'beta_0': 0.0, 'gamma_0': 0.0})


In [14]:
# Sample solutions several times from the circuit with these optimal parameters.
runs = 10000

# Define a circuit simulator.
simulator = qsimcirq.QSimSimulator(
    qsimcirq.QSimOptions(
         use_gpu=False,
         cpu_threads=8,
         verbosity=0
    )
)

# Add a measurement operator at the very end (across all qubits).
qaoa_circuit_with_measurement = qaoa_circuit + cirq.Circuit(cirq.measure(H.qubits, key='m'))

# Run the circuit a bunch.
measurement_outcomes = simulator.run(
    qaoa_circuit_with_measurement, param_resolver=optimal_parameters, repetitions=runs
)

# Save the results into a histogram-like dictionary.
results = measurement_outcomes.histogram(key='m')
print(f'Top three measurements (solution, frequency): {results.most_common(3)}')

# Pull out the solutions and their frequency in the simulation measurements, in descending order.
solutions, frequencies = zip(*results.most_common(len(results)))

Top three measurements (solution, frequency): [(7, 1302), (2, 1295), (4, 1285)]
